# LangChain 核心模块：Data Conneciton - Vector Stores

存储和搜索非结构化数据最常见的方法之一是将其嵌入并存储生成的嵌入向量，然后在查询时将非结构化查询进行嵌入，并检索与嵌入查询“最相似”的嵌入向量。

向量存储库负责为您存储已经过嵌入处理的数据并执行向量搜索。


![](https://python.langchain.com/assets/images/vector_stores-9dc1ecb68c4cb446df110764c9cc07e0.jpg)


下面以 `Chroma` 为例展示功能和用法

In [1]:
# 安装必要依赖包
!pip install chromadb


  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/f2/30/0d767083adb204422e10a5529f57a8afa9dc921f76620f11693e67b23c3a/chromadb-0.4.8-py3-none-any.whl.metadata
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for fastapi<0.100.0,>=0.95.2 from https://files.pythonhosted.org/packages/73/eb/03b691afa0b5ffa1e93ed34f97ec1e7855c758efbdcfb16c209af0b0506b/fastapi-0.99.1-py3-none-any.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c66a76653c62e894e0e3c3461453c5341ca926/posthog-3.0.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pul

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects


## 使用 Chroma 作为向量数据库，实现语义搜索


In [3]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

# 加载长文本
raw_documents = TextLoader('../../tests/state_of_the_union.txt', encoding='utf-8').load()

In [4]:
# 实例化文本分割器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

In [5]:
# 分割文本
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 164, which is longer than the specified 100
Created a chunk of size 169, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 139, which is longer than the specified 100
Created a chunk of size 181, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 129, which is longer than the specified 100
Created a chunk of size 146, which is longer than the specified 100
Created a chunk of size 136, which is longer than the specified 100
Created a chunk of size 189, which is longer than the specified 100
Created a chunk of size 215, which is longer than the specified 100
Created a chunk of size 124, which is longer than the specified 100
Created a chunk of size 118, which is longer tha

In [6]:
documents

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.', metadata={'source': '../../tests/state_of_the_union.txt'}),
 Document(page_content='Last year COVID-19 kept us apart. This year we are finally together again.', metadata={'source': '../../tests/state_of_the_union.txt'}),
 Document(page_content='Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.', metadata={'source': '../../tests/state_of_the_union.txt'}),
 Document(page_content='With a duty to one another to the American people to the Constitution.', metadata={'source': '../../tests/state_of_the_union.txt'}),
 Document(page_content='And with an unwavering resolve that freedom will always triumph over tyranny.', metadata={'source': '../../tests/state_of_the_union.txt'}),
 Document(page_content='Six days ago, Russia’s Vladimir Putin sought to shake the founda

In [14]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()


 ········


In [16]:
# 将分割后的文本，使用 OpenAI 嵌入模型获取嵌入向量，并存储在 Chroma 中
db = Chroma.from_documents(documents, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

#### 使用文本进行语义相似度搜索

In [10]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)
print(docs[0].page_content)

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


### 使用嵌入向量进行语义相似度搜索

In [11]:
# embeddings_model = OpenAIEmbeddings()
# embedding_vector = embeddings_model.embed_query(query)
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
